In [1]:
import dotenv
dotenv.load_dotenv()
import os
import weave
import wandb

In [2]:
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM, AutoTokenizer, TrainingArguments
)
from peft import LoraConfig, get_peft_model, TaskType, AutoPeftModelForCausalLM
from trl import SFTTrainer
import torch

/home/kilnaar/anaconda3/envs/gemma-tune/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
wandb.login(key = os.getenv("WANDB_API_KEY"))
run = wandb.init(entity = "june-pov", project = "tuned-use")

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for zscalersre.wandb.io to your netrc file: /home/kilnaar/.netrc
wandb: Currently logged in as: cumbel (june-pov) to https://zscalersre.wandb.io. Use `wandb login --relogin` to force relogin


In [4]:
target_adapter_name = "test-gemma-lora-adapter"

# Download adapted model

In [5]:
path = run.use_artifact("june-pov/model-registry/test-gemma-lora-adapter:latest").download()

wandb: Downloading large artifact test-gemma-lora-adapter:latest, 160.11MB. 33 files... 
wandb:   33 of 33 files downloaded.  
Done. 0:0:0.5 (331.3MB/s)


# Load model

In [6]:
ft_model = AutoPeftModelForCausalLM.from_pretrained(
    target_adapter_name,
    torch_dtype = torch.bfloat16,
    device_map = "auto",
)

tokenizer  = AutoTokenizer.from_pretrained(target_adapter_name)

Loading checkpoint shards: 100%|██████████| 2/2 [00:00<00:00,  3.96it/s]
We've detected an older driver with an RTX 4000 series GPU. These drivers have issues with P2P. This can affect the multi-gpu inference when using accelerate device_map.Please make sure to update your driver to the latest version which resolves this.


# Inference

In [7]:
@weave.op()
def analyze_quote(quote):
    prompt = f"Quote: {quote}\nAuthor:"
    return tokenizer.decode(ft_model.generate(
        **tokenizer(prompt, return_tensors="pt").to(ft_model.device),
        max_new_tokens=16)[0], skip_special_tokens=True)

In [8]:
print(analyze_quote("We have nothing to fear but fear itself"))

 (subsequent messages of this type will be suppressed)


Quote: We have nothing to fear but fear itself
Author: Franklin D. Roosevelt
Source: The New York Times, 193


In [9]:
print(analyze_quote("Early to bed and early to rise makes a man healthy, wealthy, and wise"))

Quote: Early to bed and early to rise makes a man healthy, wealthy, and wise
Author: Benjamin Franklin
Category: Health
Source: The Franklin Reader
Date: 


In [10]:
print(analyze_quote("An ounce of prevention is worth a pound of cure"))

Quote: An ounce of prevention is worth a pound of cure
Author: Benjamin Franklin
Category: Health
Source: The Franklin Reader
Date: 
